## Description
Attached you find a randomly sampled extract of some of our policies.
We want to be able to predict the "number_of_payment_faults" 
(this is the number of payment attempts which failed when we try to charge the money for the policy from the customer) in the future for each new policy.
There is a small legend in the document to describe some of the fields.
If there are open questions about the data you can't answer by yourself do not hesitate to approach us!
Feel free to use whatever tool or language you feel comfortable with to solve the task.

## Task
Create a statistical model to predict the "number_of_payment_faults" for a policy.
There is no fixed or expected outcome what we want to see from you. 
We prefer quality over speed. 
It does not only matter if your solution yields correct results, but also your overall project structure, tools used, etc.


In [699]:
import collections

import math

from datetime import datetime

import statsmodels.formula.api as smf

import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype

import numpy as np
from numpy import inf

from scipy import stats
from scipy.stats import skew, norm
from scipy.stats import randint as sp_randint

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import pyplot
%matplotlib inline
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')


from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.preprocessing import Normalizer, StandardScaler, RobustScaler
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

import lightgbm as lgb

from hyperopt import hp, tpe, STATUS_OK, Trials
from hyperopt.fmin import fmin

import warnings
warnings.filterwarnings("ignore")
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8")) #check the files available in the directory

np.random.seed(42)

## 1. Load Data

In [ ]:
df_friday = pd.read_excel('Your_Dataset.xlsx')

In [ ]:
df_friday.shape

In [ ]:
df_friday.head()

In [ ]:
df_train = df_friday[df_friday['number_of_payment_faults'].notnull()]

In [ ]:
df_test = df_friday[df_friday['number_of_payment_faults'].isnull()]

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train[df_train['number_of_payment_faults'].isnull()].shape

## 2. EDA and Data Processing

In [ ]:
df_friday.head()

In [ ]:
df_friday.info()

In [ ]:
df_friday.shape

- ### Missing Data

#### Check percentage of missing data

If more than 50% of the data is missing, we should delete the corresponding variable and pretend it never existed. 

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# missing_data.head(20)
missing_data

In [ ]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=percent.index, y=percent)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
#Since most of the observations don't have information of 'comprehensive_product', I delete this field.
df_train.drop('comprehensive_product', axis=1, inplace=True)
df_test.drop('comprehensive_product', axis=1, inplace=True)

In [ ]:
df_train.head()

### 'sf_class_tpl' 

In [ ]:
df_train['sf_class_tpl'].describe()

In [ ]:
df_train['sf_class_tpl'].unique()

In [ ]:
df_train[df_train['sf_class_tpl'].isnull()].shape

### 'sf_class_fc'

In [ ]:
df_train['sf_class_fc'].describe()

In [ ]:
df_train['sf_class_fc'].unique()

In [ ]:
df_train[df_train['sf_class_fc'].isnull()].shape

### 'policy_start'

In [ ]:
df_train['policy_start'].describe()

In [ ]:
df_train['policy_start'].unique()

In [ ]:
df_train[df_train['policy_start'].isnull()].shape

### 'tariff_type'

In [ ]:
df_train['tariff_type'].describe()

In [ ]:
df_train['tariff_type'].unique()

In [ ]:
df_train[df_train['tariff_type'].isnull()].shape

### 'type_of_insurance'

In [ ]:
df_train['type_of_insurance'].describe()

In [ ]:
df_train['type_of_insurance'].unique()

In [ ]:
df_train[df_train['type_of_insurance'].isnull()].shape

### 'payment_interval'

In [ ]:
df_train['payment_interval'].describe()

In [ ]:
df_train['payment_interval'].unique()

In [ ]:
df_train[df_train['payment_interval'].isnull()].shape

### 'insured_parties'

In [ ]:
df_train['insured_parties'].describe()

In [ ]:
df_train['insured_parties'].unique()

In [ ]:
df_train[df_train['insured_parties'].isnull()].shape

### 'profession_group'

In [ ]:
df_train['profession_group'].describe()

In [ ]:
df_train['profession_group'].unique()

In [ ]:
df_train[df_train['profession_group'].isnull()].shape

### 'age_insured_person'

In [ ]:
def report_numerical_variables(df_friday, var_name):
    print('-------------')
    print(var_name)
    print('\n')
    
    print(df_friday[var_name].describe())
    
    sns.distplot(df_friday[var_name])
    
    #skewness and kurtosis
    print("Skewness: %f" % df_friday[var_name].skew())

    df_friday[[var_name]].boxplot()
    pyplot.show()

    df_friday[[var_name]].hist()
    pyplot.show()

    print('mean: {} median: {}'.format(df_friday[var_name].mean(), df_friday[var_name].median()))

    df_friday[df_friday[var_name].isnull()].shape

    sns.distplot(df_friday[var_name], fit=norm)
    fig = plt.figure()

    # https://matplotlib.org/mpl-probscale/tutorial/closer_look_at_viz.html
    res = stats.probplot(df_friday[var_name], plot=plt)
    
    num_null = df_friday[df_friday[var_name].isnull()].shape[0]
    print('number of null rows: {}'.format(num_null))


In [ ]:
report_numerical_variables(df_train, 'age_insured_person')

### 'fc_deductible'

In [ ]:
report_numerical_variables(df_train, 'fc_deductible')

### 'pc_deductible'

In [ ]:
report_numerical_variables(df_train, 'pc_deductible')

### 'car_age_at_purchase'

In [ ]:
report_numerical_variables(df_train, 'car_age_at_purchase')

### 'car_age_contract_start'

In [ ]:
report_numerical_variables(df_train, 'car_age_contract_start')

### 'annual_mileage'

In [ ]:
report_numerical_variables(df_train, 'annual_mileage')

- ### Transforming some numerical variables that are really categorical
    - 'risk_predictor_zip_code'

In [ ]:
df_train['risk_predictor_zip_code'] = df_train['risk_predictor_zip_code'].apply(str)
df_test['risk_predictor_zip_code'] = df_test['risk_predictor_zip_code'].apply(str)

In [ ]:
df_train.info()

- ### Add new column '_na' for all the numerical variables

In [ ]:
# handle the missing values: 
# - create a col_NA column to indicate which row has NAs.
def add_col_na(df_train, target):
    for col in df_train.columns:
        if col != target and is_numeric_dtype(df_train[col]):
            col_vals = df_train[col]
            if sum(col_vals.isnull()) != 0:
                df_train[col+'_na'] = col_vals.isnull()
                df_test[col+'_na'] = df_test[col].isnull()                
    return df_train, df_test

In [ ]:
df_train, df_test = add_col_na(df_train, 'number_of_payment_faults')

- ### Skewness


Transform the skewed numeric features by taking log(feature + 1). This will make the features more normal.

https://www.kaggle.com/humananalog/xgboost-lasso/code

Other approaches:

Box Cox Transformation of (highly) skewed features

https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

In [ ]:
df_train.info()

In [ ]:
numeric_features = ['fc_deductible', 'pc_deductible', 'car_age_at_purchase', 
                    'car_age_contract_start', 'annual_mileage', 'risk_predictor_zip_code'] 

In [ ]:
skewed = df_train[numeric_features].apply(lambda x: skew(x.dropna().astype(int)))
skewed = skewed[skewed > 0.75]
skewed = skewed.index

df_train[skewed] = np.log1p(df_train[skewed])
df_test[skewed]  = np.log1p(df_test[skewed])

- ### Outliers

Some outliers could be removed, however outliers removal is note always safe. 
I decided not to delete any of the outliers, because I found there are many outliers in the dataset.
On the other hand, as indicated by https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
removing all outliers may affect badly the models if ever there were also outliers in the test data. That's why , instead of removing them all, we will just manage to make some of our models robust on them. 


- ### Scale numerical features

In [ ]:
numeric_features = ['fc_deductible', 'pc_deductible', 'car_age_at_purchase', 
                    'car_age_contract_start', 'annual_mileage'] 

scaler = StandardScaler()
scaler.fit(df_train[numeric_features])

scaled = scaler.transform(df_train[numeric_features])
for i, col in enumerate(numeric_features):
    df_train[col] = scaled[:, i]
    
scaled = scaler.transform(df_test[numeric_features])
for i, col in enumerate(numeric_features):
    df_test[col] = scaled[:, i]


- ### One-hot encode categorical features


In [ ]:
categorical_features = ['sf_class_tpl', 'sf_class_fc', 'policy_start', 'tariff_type', 'type_of_insurance', 
                        'payment_interval', 'insured_parties', 'profession_group', 'risk_predictor_zip_code'] 

In [ ]:
#http://queirozf.com/entries/one-hot-encoding-a-feature-on-a-pandas-dataframe-an-example
#https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data
dataset = pd.concat(objs=[df_train, df_test], axis=0)

for categorical_feature in categorical_features:
    dataset = pd.concat([dataset, 
                         pd.get_dummies(dataset[categorical_feature], prefix=categorical_feature, dummy_na=True)], 
                        axis=1)
    
    # now drop the original 'country' column (you don't need it anymore)
    dataset.drop([categorical_feature], axis=1, inplace=True)

train_len = len(df_train)
df_train = dataset[:train_len]
df_test = dataset[train_len:]

In [ ]:
print(df_train.columns)

- ### Convert boolean features to integer
We could also convert boolean features with one-hot encoding, if we want to us linear regression, neural networks, svm as regressors.

In [ ]:
df_train.head()

In [ ]:
bool_feature_columns = df_train.dtypes[df_train.dtypes == "bool"].index
bool_feature_columns.values

print(bool_feature_columns)

In [ ]:
for bool_feature_col in bool_feature_columns:
    df_train[bool_feature_col] = df_train[bool_feature_col].astype('int32')
    df_test[bool_feature_col] = df_test[bool_feature_col].astype('int32')

In [ ]:
df_train.head()

- ### Log-transformation of the target variable

As (linear) models work well on normally distributed data, we need to transform the variable and make it more normally distributed.

In [ ]:
# https://www.kaggle.com/humananalog/xgboost-lasso/code
# https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

df_train['number_of_payment_faults'] = df_train['number_of_payment_faults'].astype('int32')

df_target = pd.DataFrame(index = df_train.index, columns=['number_of_payment_faults'])
df_target['number_of_payment_faults'] = np.log1p(df_train['number_of_payment_faults'])

- ### Remove 'contract_nr'
'contract_nr' will not be used as a feature to train a classifier

In [ ]:
df_train_contract_nr = pd.DataFrame(index = df_train.index, columns=['contract_nr']) 
df_train_contract_nr['contract_nr'] = df_train['contract_nr']

df_test_contract_nr  = pd.DataFrame(index = df_test.index, columns=['contract_nr'])
df_test_contract_nr['contract_nr'] = df_test['contract_nr']

In [ ]:
df_train.drop(['contract_nr'], axis=1, inplace=True)
df_test.drop(['contract_nr'], axis=1, inplace=True)

In [ ]:
print("Training set size:", df_train.shape)
print("Training target set size:", df_target.shape)
print("Test set size:", df_test.shape)

- ### TODO: Remove highly correlated features

In [ ]:
#plt.figure(figsize=(15, 15))
#ax = sns.heatmap(df_train.corr(), vmax=.8, square=True, fmt='.2f', annot=True, linecolor='white', linewidths=0.01)
#plt.title('Cross correlation between variables')
#plt.show()

In [ ]:
#Correlation map to see how features are correlated with SalePrice
corrmat = df_train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

## 3. Modelling

**Reference**

https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

https://xgboost.readthedocs.io/en/release_0.72/tutorials/model.html

https://medium.com/mlreview/gradient-boosting-from-scratch-1e317ae4587d

https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/